# 1 - Voxelise geological model

Import all CSV files and interpolate each surface to a predefined grid over which to solve Darcy flow and heat flow.

In [ ]:
import numpy as np
import stripy
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.spatial import cKDTree
import cartopy.crs as ccrs
import os
import conduction

%matplotlib inline

In [ ]:
data_dir = "../Data/"

xmin, xmax = 1e99, 1e-99
ymin, ymax = 1e99, 1e-99
zmin, zmax = 1e99, 1e-99

csvfiles = []
for f in sorted(os.listdir(data_dir)):
    if f.endswith('.csv'):
        csvfiles.append(f)
        
        xyz = np.loadtxt(data_dir+f, delimiter=',', skiprows=1)
        
        xmin, ymin, zmin = np.minimum([xmin, ymin, zmin], xyz.min(axis=0))
        xmax, ymax, zmax = np.maximum([xmax, ymax, zmax], xyz.max(axis=0))
        
        print("{:35} : av depth {:10.3f} +- {:9.3f} metres".format(f, xyz[:,-1].mean(), np.std(xyz[:,-1])))

csvfiles = list(sorted(csvfiles))

print(" ")
print("x {:.3f} -> {:.3f}".format(xmin/1e3,xmax/1e3))
print("y {:.3f} -> {:.3f}".format(ymin/1e3,ymax/1e3))
print("z {:.3f} -> {:.3f}".format(zmin/1e3,zmax/1e3))


In [ ]:
## setup model resolution

# global size
Nx, Ny, Nz = 100, 100, 200

# get local grid
solver = conduction.ConductionND((xmin,ymin,zmin), (xmax,ymax,zmax), (Nx,Ny,Nz))
Xcoords, Ycoords, Zcoords = solver.grid_coords
nx, ny, nz = Xcoords.size, Ycoords.size, Zcoords.size

tree = cKDTree(solver.coords)


xq, yq = np.meshgrid(Xcoords, Ycoords)
xq_ = xq.ravel()
yq_ = yq.ravel()
tree_layer = cKDTree(np.c_[xq_,yq_])

voxel_model = np.full((nz,ny,nx), -1, dtype=np.int)
layer_mask = np.zeros(nz*ny*nx, dtype=bool)

In [ ]:
z_grid_prev = np.zeros((ny,nx))

grid_list = []

for lith, f in enumerate(csvfiles):

    # load the surface and remove duplicates
    x,y,z = np.loadtxt(data_dir+f, delimiter=',', skiprows=1, unpack=True)
    unique_xy, unique_index = np.unique(np.c_[x,y], axis=0, return_index=True)
    x, y = list(unique_xy.T)
    z = z[unique_index]

    # interpolate to grid
    mesh = stripy.Triangulation(x, y, tree=True, permute=True)
    mesh.update_tension_factors(z)
    z_grid = mesh.interpolate_to_grid(Xcoords, Ycoords, z)
    z_near, zierr = mesh.interpolate_nearest(xq_, yq_, z)
    z_near = z_near.reshape(z_grid.shape)

    # set entries outside tolerance to nearest neighbour interpolant
    ztol = 0.1*(np.percentile(z,90) - np.percentile(z,10))
    z_mask = np.logical_or(z_grid > z_near + ztol, z_grid < z_near - ztol)
    z_grid[z_mask] = z_near[z_mask]

    # avoid the gap - make sure we interpolate where we have data
    d, idx = mesh.nearest_vertices(xq_, yq_)
    dtol = np.sqrt(mesh.areas()).max()
    z_inv_mask = np.logical_or(zierr==1, d > dtol)
    z_grid.flat[z_inv_mask] = z_grid_prev.flat[z_inv_mask]

    # interpolate to voxel_model
    d, idx = tree.query(np.c_[xq_, yq_, z_grid.ravel()])
    layer_mask.fill(0)
    layer_mask[idx] = True
    i0, j0, k0 = np.where(layer_mask.reshape(nz,ny,nx))
    for i in range(i0.size):
        voxel_model[:i0[i], j0[i], k0[i]] = lith+1


    # store for next surface
    z_grid_prev = z_grid.copy()
    
    grid_list.append(z_grid.copy())
    

In [ ]:
plt.imshow(grid_list[-4], extent=[xmin,xmax,ymin,ymax])
plt.colorbar()

In [ ]:
plt.imshow(voxel_model[:,50,:], origin='lower')
plt.colorbar()

In [ ]:
h5_output = "sydney_basin.h5"

solver.save_mesh_to_hdf5(h5_output)
solver.save_field_to_hdf5(h5_output, lithology=voxel_model.ravel())
conduction.tools.generateXdmf(h5_output)